# Test instantiating PypIt parameter sets

In [1]:
# import
import os
from configobj import ConfigObj
from pypit.par import pypitpar

## General usage

In [ ]:
# To get the default parameters, declare the parameters set
# without arguments
p = pypitpar.OverscanPar()
# Use print() to get a short-form representation
print('Print output:')
print(p)
# Use the info() method to get a long-form representation
print('.info() output:')
p.info()
# Both of these can go haywire for complex constructions of
# parameter sets (sets within sets) but are well formatted
# in the simplest cases

# Use the to_config() method to output to a *.cfg file with
# comments for each item.  You have to provide a section
# name in most cases.
p.to_config('test_overscan.cfg', section_name='overscan')
# You can also use the to_config() method to just get the
# list of strings that are then written to the file.  This
# is useful when you want to reconstruct a ConfigObj.
p_cfgobj = ConfigObj(p.to_config(None, section_name='overscan',
                                 just_lines=True))
print(p_cfgobj)

## Demo of parameter defaults

The following just shows the results of the default parameter sets to show what parameters are defined and where.

### Run parameters

In [ ]:
pypitpar.RunPar()

### Reduction parameter set

The reduction parameter set is made up of 6 sub parameter sets

In [ ]:
#1
pypitpar.OverscanPar()

In [ ]:
#2
pypitpar.FlatFieldPar()

In [ ]:
#3
pypitpar.FlexurePar()

In [ ]:
#4
pypitpar.WavelengthCalibrationPar()

In [ ]:
#5
pypitpar.FluxCalibrationPar()

In [ ]:
#6
pypitpar.SkySubtractionPar()

In [ ]:
# This is the composite parameter set
pypitpar.ReducePar()

### Frame groups

Parameters used for each frame type (bias, pixelflat, etc) are abstracted to a single "Frame-Group" parameter set.  The default is 'bias'; valid frame types are shown using the static method `valid_frame_types` (see below).  The parameters used to combine frames are a subset of the Frame-Group Parameters

In [ ]:
pypitpar.CombineFramesPar()

In [ ]:
pypitpar.FrameGroupPar.valid_frame_types()

In [ ]:
pypitpar.FrameGroupPar()

### Wavelength solution

In [ ]:
pypitpar.WavelengthSolutionPar()

### Slit tracing

The slit tracing parameter set includes the PCA parameters as a separate subset

In [ ]:
pypitpar.PCAPar()

In [ ]:
pypitpar.TraceSlitsPar()

### Tilt tracing

In [ ]:
pypitpar.TraceTiltsPar()

### Object tracing

In [ ]:
pypitpar.TraceObjectsPar()

### Object extraction

I provide an object that defines any manual extractions (a bit overkill):

In [ ]:
pypitpar.ManualExtractionPar()

Then the `manual` parameter in the `ExtractObjectsPar` is a list of the `ManualExtractionPar` objects for all the manual extractions to perform.  The number of manual extractions is just the length of the list or 0 if the element is `None`.

In [ ]:
p = pypitpar.ExtractObjectsPar()
print(p)  # The printing goes a bit wrong for 'manual'
nmanual = 0 if p['manual'] is None else len(p['manual'])
print('Manual extractions to perform: {0}'.format(nmanual))

### Instrument Parameters

Similar to the `ExtractObjectsPar`, the `InstrumentPar` contains a list of other parameter sets.  In this case, it's the list of detectors.  Here's the list of detector parameters:

In [ ]:
pypitpar.DetectorPar()

Unlike `ExtractObjectsPar`, the `InstrumentPar` gives you at least one detector by default.

In [ ]:
p = pypitpar.InstrumentPar()
print(p)  # Again, the printing goes wrong because detector is a list...
print('Number of detectors: {0}'.format(len(p['detector'])))

### Instrument fits files

The `FrameFitsPar` provides the list of instrument-specific fits file parameters needed to read the data for *any* fits file from the instrument.

In [ ]:
pypitpar.FrameFitsPar()

The `keydef` parameter is a dictionary with the list of defined keywords, and the `keycheck` parameter is a dictionary with the list of values that those keywords should take for a valid fits file.  The keyword definition and checking syntax is a bit different from what currently exists:

In [ ]:
print(pypitpar.FrameFitsPar().descr['keydef'])

In [ ]:
print(pypitpar.FrameFitsPar().descr['keycheck'])

The defaults are meaningless, so here's the specific case of the Keck LRISb specifications.

In [ ]:
inp_cfg = ConfigObj('../../pypit/config/spectrographs/KECK_LRISb_spectrograph.cfg')
cfg = pypitpar._recursive_dict_evaluate(inp_cfg)

p = pypitpar.FrameFitsPar.from_dict(cfg['fits'])
print('Header keyword definitions:')
print(p['keydef'])
print('\nHeader keyword checks:')
print(p['keycheck'])

### Frame type identifications

Like the frame groupings, I've abstracted the frame type identification parameters into the `FrameIDPar` class.

In [ ]:
pypitpar.FrameIDPar()

The `fitspar` keyword points to the general parameters in the `FrameFitsPar` for the instrument.  Again, the defaults are meaningless, so here's the specific case of the bias frames for Keck LRISb.

In [ ]:
biasp = pypitpar.FrameIDPar.from_dict(p, 'bias', cfg['biasid'])
print(biasp['keycheck'])

## Putting it all together

All of the above are expected to be parameter subsets that are passed to individual methods.  Sticking with the idea of having a single object that provides all the parameters needed for a run of `pypit`, the `PypitPar` object just collects all the above objects into a single parameter set.

In [2]:
pypitpar.PypitPar()

    Parameter      Value    Default          Type  Callable
-----------------------------------------------------------
          rdx  see below  see below  ParSet, dict     False
 calibrations  see below  see below  ParSet, dict     False
standardframe  see below  see below  ParSet, dict     False
 scienceframe  see below  see below  ParSet, dict     False
      objects  see below  see below  ParSet, dict     False
      extract  see below  see below  ParSet, dict     False
  skysubtract  see below  see below  ParSet, dict     False
      flexure  see below  see below  ParSet, dict     False
    wavecalib  see below  see below  ParSet, dict     False
    fluxcalib  see below  see below  ParSet, dict     False

rdx
   Parameter    Value  Default  Type  Callable
----------------------------------------------
spectrograph     None     None   str     False
    pipeline     None     None   str     False
       ncpus        1        1   int     False
      detnum     None     None   int    

Like all the other pypit parameter sets, `PypitPar` has a `from_dict` method, but we're mostly going to want to use its `from_cfg_file` method to build the parameter set for each pypit run.

In [ ]:
help(pypitpar.PypitPar.from_cfg_file)

You can get the default parameters and print the result like this.

In [3]:
# Write the defaults
p = pypitpar.PypitPar.from_cfg_file()
p.to_config('default.cfg')

You can then read it back in.  The `expand_spectrograph` option allows you to set all the spectrograph setting using the `spectrograph` keyword in the input config file (this is `KECK_LRISb` by default).  If you just want to use what's in the config file without finding the spectrograph configuration file (because those keywords are already in the configuration file in this example), you would set `expand_spectrograph=False`.

In [4]:
# Read them in
p = pypitpar.PypitPar.from_cfg_file('default.cfg')
p['rdx']['spectrograph'] is None

True

### User-level example

I import `PypitPar` as part of the `__init__.py` setup of the `pypit.par` module, meaning you can import it directly.  I expect this is how we'll use it in the rest of the code.

In [5]:
from pypit.par import PypitPar

To set up a `PypitPar` object similar to what's currently done using the `keck_lris_blue_long_400_3400_d560.pypit` file in the dev suite, I would do the following:

In [6]:
p = PypitPar.from_cfg_file(merge_with='keck_lris_blue_long_400_3400_d560.cfg')

I actually had to comment out the definition of the flexure spectrum because I can't find it on my disk, and the declaration above faults when it can't find the file.

With the declaration above, all the default parameters are set, the spectrograph key (`KECK_LRISb`) is used to find the appropriate spectrograph definition file (`../../pypit/config/spectrographs/KECK_LRISb_spectrograph.cfg`), and both are merged with the alterations in the local `keck_lris_blue_long_400_3400_d560.cfg` file.

You can print the compiled parameters using the `to_config` method.

In [7]:
p.to_config('compiled_example.cfg')